In [ ]:

import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from sklearn.preprocessing import MinMaxScaler
from sklearn.metrics import mean_squared_error
from sklearn.neural_network import MLPRegressor
from deap import base, creator, tools, algorithms
import random
from functools import partial

df = pd.read_csv("macro_data.csv")


In [ ]:

def create_lag_features(df, lags):
    df_lagged = pd.DataFrame()
    for col, lag in lags.items():
        df_lagged[f"{col}(lag)"] = df[col].shift(lag)
    df_lagged["IPC(+1)"] = df["IPC"].shift(-1)
    df_lagged.dropna(inplace=True)
    return df_lagged.reset_index(drop=True)

lags = {"IOC": 0, "IPC": 0, "KVVE": 7, "M0": 7, "M2": 7}
data = create_lag_features(df, lags)
data.head()


In [ ]:

results = []
true_vals = []
pred_vals = []


if "FitnessMin" not in creator.__dict__:
    creator.create("FitnessMin", base.Fitness, weights=(-1.0,))
if "Individual" not in creator.__dict__:
    creator.create("Individual", list, fitness=creator.FitnessMin)

for i in range(10):
    window = data.iloc[i:i+12]
    X = window.drop("IPC(+1)", axis=1).values
    y = window["IPC(+1)"].values

    scaler_x = MinMaxScaler()
    scaler_y = MinMaxScaler()
    X_scaled = scaler_x.fit_transform(X)
    y_scaled = scaler_y.fit_transform(y.reshape(-1, 1)).flatten()

    mlp = MLPRegressor(hidden_layer_sizes=(5,), activation='relu', max_iter=1, warm_start=True)
    mlp.fit(X_scaled, y_scaled)

    n_weights = sum(w.size for w in mlp.coefs_ + mlp.intercepts_)
    toolbox = base.Toolbox()
    toolbox.register("attr_float", lambda: random.uniform(-1.0, 1.0))
    toolbox.register("individual", tools.initRepeat, creator.Individual, toolbox.attr_float, n=n_weights)
    toolbox.register("population", tools.initRepeat, list, toolbox.individual)

    def eval_nn(individual):
        start = 0
        i_weights = []
        for coef in mlp.coefs_ + mlp.intercepts_:
            shape = coef.shape
            size = np.prod(shape)
            new_weights = np.array(individual[start:start + size]).reshape(shape)
            i_weights.append(new_weights)
            start += size
        mlp.coefs_ = i_weights[:len(mlp.coefs_)]
        mlp.intercepts_ = i_weights[len(mlp.coefs_):]
        y_pred = mlp.predict(X_scaled)
        return (mean_squared_error(y_scaled, y_pred),)

    toolbox.register("evaluate", eval_nn)
    toolbox.register("mate", tools.cxTwoPoint)
    toolbox.register("mutate", tools.mutGaussian, mu=0, sigma=0.5, indpb=0.2)
    toolbox.register("select", tools.selTournament, tournsize=3)

    pop = toolbox.population(n=20)
    hof = tools.HallOfFame(1)
    algorithms.eaSimple(pop, toolbox, cxpb=0.7, mutpb=0.3, ngen=20, halloffame=hof, verbose=False)

    
    start = 0
    i_weights = []
    for coef in mlp.coefs_ + mlp.intercepts_:
        shape = coef.shape
        size = np.prod(shape)
        new_weights = np.array(hof[0][start:start + size]).reshape(shape)
        i_weights.append(new_weights)
        start += size

    mlp.coefs_ = i_weights[:len(mlp.coefs_)]
    mlp.intercepts_ = i_weights[len(mlp.coefs_):]

    y_pred_scaled = mlp.predict(X_scaled)
    y_pred_real = scaler_y.inverse_transform(y_pred_scaled.reshape(-1, 1)).flatten()
    mse_real = mean_squared_error(y, y_pred_real)

    results.append({
        "Window": f"{i+1}-{i+12}",
        "Real IPC(+1)": y[-1],
        "Predicted IPC(+1)": y_pred_real[-1],
        "MSE": mse_real
    })
    true_vals.append(y[-1])
    pred_vals.append(y_pred_real[-1])

df_results = pd.DataFrame(results)
df_results


In [ ]:

print("Average MSE:", np.mean(df_results["MSE"]))
print("Maximum:", np.max(df_results["MSE"]))
print("Minimum:", np.min(df_results["MSE"]))


In [ ]:

plt.figure(figsize=(10, 5))
plt.plot(true_vals, label="Real IPC(+1)")
plt.plot(pred_vals, label="Predicted IPC(+1)")
plt.title("Forecast 13 points in each window (1–10)")
plt.xlabel("Window")
plt.ylabel("IPC")
plt.legend()
plt.grid(True)
plt.show()


In [ ]:



X_full = data.drop("IPC(+1)", axis=1).values
y_full = data["IPC(+1)"].values

scaler_x_full = MinMaxScaler()
scaler_y_full = MinMaxScaler()
X_full_scaled = scaler_x_full.fit_transform(X_full)
y_full_scaled = scaler_y_full.fit_transform(y_full.reshape(-1, 1)).flatten()

mlp_full = MLPRegressor(hidden_layer_sizes=(5,), activation='relu', max_iter=1, warm_start=True)
mlp_full.fit(X_full_scaled, y_full_scaled)

n_weights = sum(w.size for w in mlp_full.coefs_ + mlp_full.intercepts_)
toolbox = base.Toolbox()
toolbox.register("attr_float", lambda: random.uniform(-1.0, 1.0))
toolbox.register("individual", tools.initRepeat, creator.Individual, toolbox.attr_float, n=n_weights)
toolbox.register("population", tools.initRepeat, list, toolbox.individual)

def eval_full(individual):
    start = 0
    i_weights = []
    for coef in mlp_full.coefs_ + mlp_full.intercepts_:
        shape = coef.shape
        size = np.prod(shape)
        new_weights = np.array(individual[start:start + size]).reshape(shape)
        i_weights.append(new_weights)
        start += size
    mlp_full.coefs_ = i_weights[:len(mlp_full.coefs_)]
    mlp_full.intercepts_ = i_weights[len(mlp_full.coefs_):]
    y_pred = mlp_full.predict(X_full_scaled)
    return (mean_squared_error(y_full_scaled, y_pred),)

toolbox.register("evaluate", eval_full)
toolbox.register("mate", tools.cxTwoPoint)
toolbox.register("mutate", tools.mutGaussian, mu=0, sigma=0.5, indpb=0.2)
toolbox.register("select", tools.selTournament, tournsize=3)

pop = toolbox.population(n=20)
hof = tools.HallOfFame(1)
algorithms.eaSimple(pop, toolbox, cxpb=0.7, mutpb=0.3, ngen=20, halloffame=hof, verbose=False)


start = 0
i_weights = []
for coef in mlp_full.coefs_ + mlp_full.intercepts_:
    shape = coef.shape
    size = np.prod(shape)
    new_weights = np.array(hof[0][start:start + size]).reshape(shape)
    i_weights.append(new_weights)
    start += size

mlp_full.coefs_ = i_weights[:len(mlp_full.coefs_)]
mlp_full.intercepts_ = i_weights[len(mlp_full.coefs_):]

y_pred_full_scaled = mlp_full.predict(X_full_scaled)
y_pred_full = scaler_y_full.inverse_transform(y_pred_full_scaled.reshape(-1, 1)).flatten()
mse_full_model = mean_squared_error(y_full, y_pred_full)
print("MSE of the full model:", mse_full_model)


In [ ]:

# === MSE graph by windows ===
plt.figure(figsize=(10, 4))
plt.plot(df_results["Window"], df_results["MSE"], marker='o', linestyle='-')
plt.xticks(rotation=45)
plt.grid(True)
plt.title("MSE for each window")
plt.xlabel("Window")
plt.ylabel("MSE")
plt.tight_layout()
plt.show()
